In [32]:
#housekeeping and essential imports
import pandas as pd
import os
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1200)
pd.set_option ('display.max_colwidth', 70)

In [13]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
os.chdir(os.path.join(os.path.expanduser("~"),"GoogleDrive/NYU/interview/rerun_match/"))
DIR = os.getcwd()
DIR

'/Volumes/Storage/chaodai/GoogleDrive/NYU/interview/rerun_match'

### Step 1: read raw data files, filter, then save filtered data in tab delimited text file

In [ ]:
%load /Users/chao/GoogleDrive/NYU/interview/rerun_match/match.py

---

### Step 2: Find reads that match to reference

In [21]:
ls ~/GoogleDrive/NYU/interview/rerun_match/

F231_PAS_Rank.txt   F231_matched.txt    F4175_filtered.txt  UTR_filtered.txt
F231_filtered.txt   F4175_PAS_Rank.txt  F4175_matched.txt   match.py


In [3]:
DIR=os.getcwd()
DIR

'/Volumes/Storage/chaodai/GoogleDrive/NYU/interview/rerun_match'

In [4]:
F2_match = pd.read_csv(os.path.join(DIR, 'F231_matched.txt'), sep='\t')
F4_match = pd.read_csv(os.path.join(DIR, 'F4175_matched.txt'), sep='\t')

### _plotting_

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')
from scipy import stats

In [9]:
p_data4 = findPAS(F4_match)

In [10]:
p_data2 = findPAS(F2_match)

In [29]:
p_data4[p_data4.ref_index.isin([99,111])].head(2)

,read_index,read_start,read_end,read_id,ref_index,ref_start,ref_end,gene_id,bin_val
46973,75687,106712589,106715264,50586bd6-c324-4570-9214-e9d9037fc6d0,99,106712589,106716955,NM_001962.2_utr3_0_0_chr5_106712590_r,106712596
46974,75688,106712589,106715227,eb09745b-9870-4302-8b0f-8ee2e5f64718,99,106712589,106716955,NM_001962.2_utr3_0_0_chr5_106712590_r,106712596


In [11]:
p_data4.to_csv(os.path.join(DIR, "F4175_plot.txt"), header=True, index=False, sep="\t")
p_data2.to_csv(os.path.join(DIR, "F231_plot.txt"), header=True, index=False, sep="\t")

In [47]:
stats.describe(p_data4.bin_val)

DescribeResult(nobs=224971, minmax=(204896, 180684296), mean=133009301.74296242, variance=1500886844198625.5, skewness=-1.0685660797367276, kurtosis=0.8364689535238741)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(30,6))
ax1.hist(p_data4[p_data4.ref_index.isin([99,111,112])].bin_val, bins=p_data4.bin_val )
plt.plot()

### _Binning - finding PAS_

In [6]:
def findPAS(df):
    """find PAS for each gene in 3' UTR. df must be a cleaned and matched dataframe """
    # set up bins (aka. potential PAS)
    bins = np.arange(min(df['read_start']), max(df['read_start'])+20, 20) # using window size = 20 bp
    # categorize read_starts into respective bins
    df['bin_order'] = np.digitize(df['read_start'], bins) #bin_order is only used to get bin_value, aka PAS
    # list PAS (bin value) (bins that have read_start fall into)
    df['bin_val'] = df['bin_order'].apply(lambda x: bins[x])
    df.drop(['bin_order'], axis=1, inplace=True) # drop column "bin_order" no longer needed
    return df

In [7]:
# Final PAS data table for F4175: | gene | PAS | number of reads per PAS | use this table to calculate relative frequency of reads that 
F4_gene_pas_count = findPAS(F4_match).groupby(['ref_index', 'gene_id', 'bin_val'])['bin_val'].count() #get a pd.series of PAS counts
F4_gene_pas_count = pd.DataFrame(F4_gene_pas_count) # convert pd.series to dataframe
F4_gene_pas_count.columns=['bin_counts'] # bin_counts are the number of reads for each PAS (given the level of index)
F4_denom = pd.DataFrame(F4_gene_pas_count.groupby(level=[0,1]).sum()).reset_index() # calculate total number of reads per gene
F4_gene_pas_count.reset_index(inplace=True) # reset index

#calculate relative frequency per PAS
F4_gene_pas_count = F4_gene_pas_count.merge(F4_denom[['ref_index', 'bin_counts']], how='left', on=['ref_index']) # join on ref_index (aka gene_id) to get total reads per geme
F4_gene_pas_count['frequency'] = F4_gene_pas_count['bin_counts_x'] / F4_gene_pas_count['bin_counts_y'] # total reads per PAS of gene / total reads per gene
F4_gene_pas_count.columns = ['ref_index', 'gene_id', 'PAS', 'reads_in_PAS', 'reads_in_gene', 'PAS_freq'] # change column names
F4_gene_pas_count['rank'] = F4_gene_pas_count.groupby(['ref_index','gene_id'])['reads_in_PAS'].rank(method='max', ascending=False)  # rank number of reads per PAS per gene (use top 3)


# Final PAS data table for F231: | gene | PAS | number of reads per PAS | use this table to calculate relative frequency of reads that 
F2_gene_pas_count = findPAS(F2_match).groupby(['ref_index', 'gene_id', 'bin_val'])['bin_val'].count() #get a pd.series of PAS counts
F2_gene_pas_count = pd.DataFrame(F2_gene_pas_count) # convert pd.series to dataframe
F2_gene_pas_count.columns=['bin_counts'] # bin_counts are the number of reads for each PAS (given the level of index)
F2_denom = pd.DataFrame(F2_gene_pas_count.groupby(level=[0,1]).sum()).reset_index() # calculate total number of reads per gene
F2_gene_pas_count.reset_index(inplace=True) # reset index

#calculate relative frequency per PAS
F2_gene_pas_count = F2_gene_pas_count.merge(F2_denom[['ref_index', 'bin_counts']], how='left', on=['ref_index']) # join on ref_index (aka gene_id) to get total reads per geme
F2_gene_pas_count['frequency'] = F2_gene_pas_count['bin_counts_x'] / F2_gene_pas_count['bin_counts_y'] # total reads per PAS of gene / total reads per gene
F2_gene_pas_count.columns = ['ref_index', 'gene_id', 'PAS', 'reads_in_PAS', 'reads_in_gene', 'PAS_freq'] # change column names
F2_gene_pas_count['rank'] = F2_gene_pas_count.groupby(['ref_index','gene_id'])['reads_in_PAS'].rank(method='max', ascending=False)  # rank number of reads per PAS per gene (use top 3)



In [477]:
# write to file
F4_gene_pas_count.to_csv(os.path.join(DIR, "F4175_PAS_Rank.txt"), index=False, header=True, sep='\t')
F2_gene_pas_count.to_csv(os.path.join(DIR, "F231_PAS_Rank.txt"), index=False, header=True, sep='\t')

In [33]:
# check f2_gene_pas_count
F2_gene_pas_count[F2_gene_pas_count.reads_in_gene>200].head()

,ref_index,gene_id,PAS,reads_in_PAS,reads_in_gene,PAS_freq,rank
425,99,NM_001962.2_utr3_0_0_chr5_106712590_r,106712596,23,965,0.023834,8.0
426,99,NM_001962.2_utr3_0_0_chr5_106712590_r,106712616,29,965,0.030052,5.0
427,99,NM_001962.2_utr3_0_0_chr5_106712590_r,106712636,2,965,0.002073,75.0
428,99,NM_001962.2_utr3_0_0_chr5_106712590_r,106712656,1,965,0.001036,110.0
429,99,NM_001962.2_utr3_0_0_chr5_106712590_r,106712676,1,965,0.001036,110.0


### _Compare F231 and F4175 PAS_

In [34]:
F2_gene_pas_count[F2_gene_pas_count.PAS.isin([16662036])]
F4_gene_pas_count[F4_gene_pas_count.PAS.isin([16662036])]

,ref_index,gene_id,PAS,reads_in_PAS,reads_in_gene,PAS_freq,rank
0,6,NM_012334.2_utr3_0_0_chr5_16662016_r,16662036,4,35,0.114286,2.0


,ref_index,gene_id,PAS,reads_in_PAS,reads_in_gene,PAS_freq,rank
0,6,NM_012334.2_utr3_0_0_chr5_16662016_r,16662036,5,42,0.119048,2.0


In [26]:
joined_PAS = pd.merge(F2_gene_pas_count, F4_gene_pas_count, how="outer", on=['gene_id', 'PAS'])

In [27]:
len(joined_PAS)
len(F2_gene_pas_count)
len(F4_gene_pas_count) # difference in lenth shows many PAS are in one sites but not in another

19014

12981

13979

### Comparing F231 and F4175

The following joined_PAS dataframe shows __x: from F231, y: from F4175__

In [28]:
joined_PAS

,ref_index_x,gene_id,PAS,reads_in_PAS_x,reads_in_gene_x,PAS_freq_x,rank_x,ref_index_y,reads_in_PAS_y,reads_in_gene_y,PAS_freq_y,rank_y
0,6.0,NM_012334.2_utr3_0_0_chr5_16662016_r,16662036,4.0,35.0,0.114286,2.0,6.0,5.0,42.0,0.119048,2.0
1,6.0,NM_012334.2_utr3_0_0_chr5_16662016_r,16663476,2.0,35.0,0.057143,5.0,6.0,3.0,42.0,0.071429,4.0
2,6.0,NM_012334.2_utr3_0_0_chr5_16662016_r,16664196,1.0,35.0,0.028571,16.0,NaN,NaN,NaN,NaN,NaN
3,6.0,NM_012334.2_utr3_0_0_chr5_16662016_r,16664256,1.0,35.0,0.028571,16.0,NaN,NaN,NaN,NaN,NaN
4,6.0,NM_012334.2_utr3_0_0_chr5_16662016_r,16664416,1.0,35.0,0.028571,16.0,NaN,NaN,NaN,NaN,NaN
5,6.0,NM_012334.2_utr3_0_0_chr5_16662016_r,16665416,13.0,35.0,0.371429,1.0,6.0,16.0,42.0,0.380952,1.0
6,6.0,NM_012334.2_utr3_0_0_chr5_16662016_r,16665436,3.0,35.0,0.085714,3.0,6.0,3.0,42.0,0.071429,4.0
7,6.0,NM_012334.2_utr3_0_0_chr5_16662016_r,16665456,1.0,35.0,0.028571,16.0,6.0,1.0,42.0,0.023810,18.0
8,6.0,NM_012334.2_utr3_0_0_chr5_16662016_r,16665556,1.0,35.0,0.028571,16.0,NaN,NaN,NaN,NaN,NaN
9,6.0,NM_012334.2_utr3_0_0_chr5_16662016_r,16665776,1.0,35.0,0.028571,16.0,NaN,NaN,NaN,NaN,NaN


---

## non-relevant below

In [169]:
# get counts of each unique reads
def count_reads(matched_file):
    """read matched file of the reads, then count matches of each read"""
    df = pd.read_csv(matched_file, sep='\t')
    # set group by columns
    L_groupby = ['read_index', 'read_start', 'read_end', 'read_id']
    # change column names to avoid confusion
    count_cols = ['read_index_count', 'read_start_count', 'read_end_count', 'read_id_count', 'ref_index_count', 'ref_start_count', 'ref_end_count', 'gene_id_count']
    df = df.groupby(L_groupby).nunique() # groupby to count matches for each read
    df.columns = count_cols #change column names
    df.reset_index(inplace=True) # move indeces into columns
    return df

In [170]:
DIR = '/Volumes/Storage/chaodai/GoogleDrive/NYU/interview'

In [173]:
F2_plot = count_reads(os.path.join(DIR, "F231_matched.txt"))

In [176]:
F4_plot = count_reads(os.path.join(DIR, "F4175_matched.txt"))

In [230]:
DIR2= "/Volumes/Storage/chaodai/GoogleDrive/NYU/interview/rerun_match/"

In [232]:
ls /Volumes/Storage/chaodai/GoogleDrive/NYU/interview/rerun_match/

F231_filtered.txt   F4175_filtered.txt  UTR_filtered.txt    match.py


In [242]:
df4 = pd.read_csv(os.path.join(DIR2, "F4175_filtered.txt"), sep="\t")

In [243]:
df4.drop(df4[df4.strand.isin(["+"])].index, inplace=True)

In [244]:
df4.to_csv(os.path.join(DIR2, "F4175_filtered.txt"), header=True, sep="\t", index=False)

In [246]:
del(df)

239285